# Topological Sorting Algorithm

## Introduction

Topological sort has more and more usage in real world and data science field now. For instance, not only the build system, it can also do instruction scheduling, deciding the order of load tables with foreign keys in databases, ordering the formula cell evaluation in spreadsheets, and so on. One of the famous application is that apt-get use topological sorting to get the best sequence in which a set of packages can be installed.

This tutorial will provide a brief introduction for Topological sorting. In the directed graph, topological sorting is the linear ordering of the whole vertexes that for every edge uv, vertex u comes before vertex v in the ordering. In addition, there may have multiple topological sorting result for a directed graph, and topological sorting can not use in the graph which is not directed graph.

For instance, 

<img src="https://www.csie.ntu.edu.tw/~b99204007/CMU/PDS/Tutorial/ts_ex1.pdf">


We have a directed graph shown above. One topological ordering result could be A->B->C->E->D->F, another ordering could be B->A->C->D->E->F. So the point here is that vertex C is never comes before vertex A and vertex B in the ordering, vertex E should never comes before C, and F should never comes before E and D, and so on. One of the application of topological sorting in data science is the usage of dependency relationship between each vertex. Suppose this graph is the realationship of vertexes, and we can see that C has dependency on A and B, and D has dependency on C, and so on. So if we use the topological sorting to create one of the ordering to know the ordering of each vertexes, and then solve the issue by dependency in this order. Like one realistic example usage in Twitter data analytics, we can use topological sorting algorithm to find all the relationship between follower and followee on Twitter.

## Algorithm 

There are two main algorithm people used to implement topological sort: *Depth First Search* and *Kahn's algorithm*. We'll briefly introduce these two algorithm below and utilize one simple example about course schduling to help you understand the whole concept easier!


### Depth First Search (DFS)

Using DFS is a straight forward way for us to approach topological sort. We are going to modify DFS and find the topological sorting of a graph. In DFS, we'll start from a vertex, store the vertex and then call DFS recursively to its adjacent. The algorithm with the loops go through each vertex and terminates when it visits the vertex which has already been visited or there is no outgoing adjacent vertex.

Lets take a briefly look at how the Depth First Search algorithm works with pseudocode:
~~~~
Result <- Empty list, and will store the sorted nodes

while any unmarked nodes 
    select one of unmarked node n
    visit(n) 

function visit(node n)
    if n has been temporary marked 
        stop
    else
        mark n temporarily
        for each node m which is adjacent to n with an edge from n to m 
            visit(m)
        unmark n temporarily
        mark n permanently
        add n to head of Result
~~~~


#### Topological Sorting with Depth First Search 

By using DFS in topological sort, we use the concept like stack with the recursive call. We don't store the vertex to answer immediately, however, we will recursively call  the DFS topological sorting function for all of the adjacent vertices, push all of them to stack. And store the answer with the order of the content of the stack.




#### Example for Topological Sorting with Depth First Search

Let's take one real-problem example. Suppose we're going to take several courses. However, there may have the prerequisites for each courses. For instance, in order to take the course 0, you have to take the course 1 first, which is expressed as: [0, 1]

We will know the total number of courses as well as the prerequisites pair which is represented as a list. We can use the topological sorting algorithm to know the correct order to take the whole courses.

*dfs()* function will traverse from one vertex to all adjacent vertex and check whether the vertex has been visited or not. If we've visited the vertex, means this path is end so return false directly, otherwise do the recursion by calling dfs() function to this adjacent vertex again until we find there is no adjacent vertex we haven't visited, then this vertex is the course we can take now.

In [1]:
from collections import defaultdict

def find_course_order(course_cnt, prerequisites):
    """
    Find the order of the course to take

    Args:
        course_cnt: integer, the total amount of the courses can take
        prerequisites: List[List[int]], the list of pair of prerequisite for the course

    Returns:
        List[int]: the order of the course to take
    """

    # Initialized the prerequisites table.
    # With the dict key corresponding to a list of it's prerequisite courses.
    table = defaultdict(list)
    for a, b in prerequisites:
        table[b] += [a]

    # Initialized the unvisited set.
    unvisited = set()
    for course in range(course_cnt):
        unvisited.add(course)

    visited = set()
    result = list()

    while unvisited:
        is_cyclic = dfs(table, unvisited.pop(), unvisited, visited, result)
        if is_cyclic: 
            print "No result because of the cycle existed"
            return []

    result = result[::-1] # Reversed the whole course list for the correct order.
    return result

def dfs(table, course, unvisited, visited, result):
    if course in visited: 
        if course in result: 
            return False
        else: 
            return True

    unvisited.discard(course) # Removed the course from unvisited set.
    visited.add(course) # Put the course to the visited set.
    for pre in table[course]:
        is_cyclic = dfs(table, pre, unvisited, visited, result)
        if is_cyclic: 
            return True
    result += [course]
    return False

With an example of dfs topological sorting algorithm shown above, now we take three small test cases to see what happens, and all three test cases code also provided below:

From example one, if there is only two courses and the prerequisites is [1, 0], means we can take the course 0 then course 1 with the result order [0, 1]. 

Second example: If the total number of courses are 4 (course_cnt = 4), and the there are several prerequisite pairs as [1, 0], [2, 0], [3, 1], and [3, 2], such as we should take course 0 before we take course 1, should take course 0 before we take course 2, and so on. And finally we can get the result list like [0, 2, 1, 3], means one of the acceptable order of taking the whole courses is 0, 2, 1 and 3.

The third example is the cycle exception, means if there is any cycle exist (0->1->3->0) in the graph, we can not have the correct order result if we still need to take all courses. So we'll print the exception *"No result because of the cycle existed"* and return the empty list.


In [2]:
# AUTOLAB_IGNORE_START
course_cnt = 2
prerequisites = []
prerequisites.append([1, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 1
course_order_result = find_course_order(course_cnt, prerequisites)
print course_order_result # Should get the result [0, 1]

course_cnt = 4
prerequisites = []
prerequisites.append([1, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 1
prerequisites.append([2, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 2
prerequisites.append([3, 1]) # Create the edge/prerequisite in the graph that we should take 1 before we take 3
prerequisites.append([3, 2]) # Create the edge/prerequisite in the graph that we should take 2 before we take 3
course_order_result = find_course_order(course_cnt, prerequisites)
print course_order_result # Should get the result [0, 2, 1, 3]

course_cnt = 4
prerequisites = []
prerequisites.append([1, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 1
prerequisites.append([2, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 2
prerequisites.append([3, 1]) # Create the edge/prerequisite in the graph that we should take 1 before we take 3
prerequisites.append([0, 3]) # Create the edge/prerequisite in the graph that we should take 3 before we take 0
course_order_result = find_course_order(course_cnt, prerequisites)
print course_order_result # Should print the exception and return the empty list
# AUTOLAB_IGNORE_STOP

[0, 1]
[0, 2, 1, 3]
No result because of the cycle existed
[]


### Kahn's algorithm

Another algorithm for topological sort is using Kahn's algorithm. It will sovlve the problems based on the in-degree (count of incoming edges) to each vertex with the same order of topological sort. By given a graph, we can find the vertices which has no incoming edges and see them as the start point as well as put them into a set S. Then each time we process the node in the set S to removing all the out-going edges. In the meanwhile with this step, we can continuously produce more nodes which has no incoming edges and add them to the set S until we find all of the node. 

Note: There must have at least one node with no incoming edge at the beginning.

We can see the simple pseudocode as below:

~~~~
Result <- Empty list, and will store the sorted nodes
S <- Set of nodes with no incoming edges
NodeCnt <- total number of nodes

while S is not empty 
    remove random one node n from S
    add n to the tail of Result list
    
    for each node m in the graph which contains an edge e from n to m do
        remove edge e from the graph
        if m has no in-coming edges anymore
            add m into set S

if size of Result set is equals to NodeCnt
    return Result (a topologically sorted order)
else 
    return error (graph has at least one cycle)

    
~~~~





#### Example for Kahn's algorithm

Let's try the similar real-problem example as the instance used in dfs above. We still have two input parameters: the amount of the courses and the prerequisites pairs expressed as [0, 1], meaning that you should take the course 1 before taking the course 0.


In [3]:
def find_course_order_with_kahn(course_cnt, prerequisites):
    """
    Find the order of the course to take

    Args:
        course_cnt: integer, the total amount of the courses can take
        prerequisites: List[List[int]], the list of pair of prerequisite for the course

    Returns:
        List[int]: the order of the course to take
    """
    
    # Initialized the prerequisite count list for each course.
    prerequisites_cnt_list = []
    for i in xrange(course_cnt):
        prerequisites_cnt_list.append(0)
    
    # Store the number of prerequisites course rather than all the courses.
    for a, b in prerequisites:
        prerequisites_cnt_list[a] += 1
        
    queue = []
    result = []

    # Grab all course with no prerequisites course at the beginning.
    for i in xrange(course_cnt):
        if prerequisites_cnt_list[i] == 0:
            queue.append(i)

    while len(queue) != 0:
        course = queue.pop()
        result.append(course)
        for i in xrange(len(prerequisites)):
            if prerequisites[i][1] == course:
                prerequisites_cnt_list[prerequisites[i][0]] -= 1
                if prerequisites_cnt_list[prerequisites[i][0]] == 0:
                    queue.append(prerequisites[i][0])

    if len(result) == course_cnt:
        return result
    else:
        print "No result because of the cycle existed"
        return []


We use the same three test cases with dfs topological sorting algorithm for you to be easier understand and compare these two algorithms. Test cases shown as below:

We can see that, the first two test cases can get results [0, 1] and [0, 2, 1, 3] correspondingly, meaning one of the order we can take the whole courses. And the third example will get print out exception because of the cycle existed. We can see the detail description and output as below:

In [4]:
# AUTOLAB_IGNORE_START
course_cnt = 2
prerequisites = []
prerequisites.append([1, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 1
course_order_result = find_course_order_with_kahn(course_cnt, prerequisites)
print course_order_result # Should get the result [0, 1]

course_cnt = 4
prerequisites = []
prerequisites.append([1, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 1
prerequisites.append([2, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 2
prerequisites.append([3, 1]) # Create the edge/prerequisite in the graph that we should take 1 before we take 3
prerequisites.append([3, 2]) # Create the edge/prerequisite in the graph that we should take 2 before we take 3
course_order_result = find_course_order_with_kahn(course_cnt, prerequisites)
print course_order_result # Should get the result [0, 2, 1, 3]

course_cnt = 4
prerequisites = []
prerequisites.append([1, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 1
prerequisites.append([2, 0]) # Create the edge/prerequisite in the graph that we should take 0 before we take 2
prerequisites.append([3, 1]) # Create the edge/prerequisite in the graph that we should take 1 before we take 3
prerequisites.append([0, 3]) # Create the edge/prerequisite in the graph that we should take 3 before we take 0
course_order_result = find_course_order_with_kahn(course_cnt, prerequisites)
print course_order_result # Should get the result [0, 2, 1, 3]
# AUTOLAB_IGNORE_STOP

[0, 1]
[0, 2, 1, 3]
No result because of the cycle existed
[]


## How about the bigger test cases

Let's try another bigger example here: 

If we have 10 courses in total with the prerequisites as [5, 8], [3, 5], [1, 9], [4, 5], [0, 2], [1, 9], [7, 8], and [4, 9], we can get the vertices relationship as the below graph. 

<img src="https://www.csie.ntu.edu.tw/~b99204007/CMU/PDS/Tutorial/ts_ex2.pdf">

We can see that there are four free courses 8, 9, 2 and 6 we can start. In other words, we can have multiple order to take the whole courses if there is no cycle in the graph. In addition, we can get one reasonable result like *[9, 1, 8, 7, 5, 4, 3, 6, 2, 0]* after running our algorithm. More detail description and output can be seen as below testing code:


In [5]:
# AUTOLAB_IGNORE_START

course_cnt = 10
prerequisites = []
prerequisites.append([5, 8]) # Create the edge/prerequisite in the graph that we should take 8 before we take 5
prerequisites.append([3, 5]) # Create the edge/prerequisite in the graph that we should take 5 before we take 3
prerequisites.append([1, 9]) # Create the edge/prerequisite in the graph that we should take 9 before we take 1
prerequisites.append([4, 5]) # Create the edge/prerequisite in the graph that we should take 5 before we take 4
prerequisites.append([0, 2]) # Create the edge/prerequisite in the graph that we should take 2 before we take 0
prerequisites.append([1, 9]) # Create the edge/prerequisite in the graph that we should take 9 before we take 1
prerequisites.append([7, 8]) # Create the edge/prerequisite in the graph that we should take 8 before we take 7
prerequisites.append([4, 9]) # Create the edge/prerequisite in the graph that we should take 9 before we take 4
course_order_result = find_course_order_with_kahn(course_cnt, prerequisites)
print course_order_result # Should get the result [9, 1, 8, 7, 5, 4, 3, 6, 2, 0]

# AUTOLAB_IGNORE_STOP

[9, 1, 8, 7, 5, 4, 3, 6, 2, 0]


## Huge test cases similar to the real world

Since we can not get the real course data from CMU, we do some mimic with some huge test cases. Here we provide two sample big test cases with 800 and 2000 courses correspondingly for you to feel how it should work in the real world. You can get the raw test case data from below links. In each test cases file, the first line is the number of courses as the first input parameter, and the second line is the prerequisite list as the second parameter.

Test cases:

(1) [800 courses test case](https://www.csie.ntu.edu.tw/~b99204007/CMU/PDS/Tutorial/TestCase/tp_test_800.txt)

(2) [2000 courses test case](https://www.csie.ntu.edu.tw/~b99204007/CMU/PDS/Tutorial/TestCase/tp_test_2000.txt)


Sample answer:

(1) [800 courses sample answer](https://www.csie.ntu.edu.tw/~b99204007/CMU/PDS/Tutorial/TestCase/tp_test_800_ans.txt)

~~~

[795,785,779,774,757,748,745,734,732,700,699,685,675,673,662,660,652,642,616,614,598,590,543,566,542,
540,539,536,523,521,520,516,515,511,494,490,482,480,479,478,474,468,464,463,458,456,455,452,451,450,
...
...
641,744,388,726,318,671,528,535,742,654,657,719,753,713,718,750,4,378,793,574,735,57,296,397,563,443,
741,418,556,613,376,484,3,110,747,472,304,393,798,778,777,197,2,697,634,791,1,396,797,0,547,555,620]

~~~

(2) [2000 courses sample answer](https://www.csie.ntu.edu.tw/~b99204007/CMU/PDS/Tutorial/TestCase/tp_test_2000_ans.txt)

~~~

[643,1143,593,1969,960,1878,654,1765,1045,1542,74,1779,1498,265,1832,1416,1276,1373,637,978,140,1190,
761,745,982,1024,1624,782,1846,712,1047,721,1479,1650,148,1417,158,1032,553,1780,1597,632,107,1152,
...
...
1012,99,1628,504,1442,1619,1882,1380,989,1124,673,547,276,281,1977,1208,506,800,731,653,335,892,1431,
1252,1503,766,1535,662,342,401,909,468,463,400,822,1273,913,341,1806,635,616,1379,656,3,719,1232,995]

~~~


## Conclusion

Topological sorting algorithm has been used in data science and more and more fields now. Although its concept is easy and mainly used for scheduling between jobs, the application today is broad and critical. For example, ordering of formula cell evaluation in spreadsheets, deciding the order of load tables with foreign keys in databases, logic synthesis, data serialization, determining the symbol dependencies, analizing the follower/followee relationship on Twitter, and so on. The apt-get application is one of the famous usage of topological sorting algorithm. Although topological sorting algorithm may just be a tiny part of what we should learn and understand in data science field, it still be useful and help you be familiar with not only a single algorithm but more about how to analyze graph and retrieve data in the future.


## References:

[1] [Advanced Packaging Tool](https://en.wikipedia.org/wiki/Advanced_Packaging_Tool)

[2] [Book "Introduction to Algorithms"](https://en.wikipedia.org/wiki/Introduction_to_Algorithms)

[3] [Depth-first search with topological sorting algorithm on Wikipedia](https://en.wikipedia.org/wiki/Topological_sorting#Depth-first_search)

[4] [Kahn's algorithm on Wikipedia](https://en.wikipedia.org/wiki/Topological_sorting#Kahn.27s_algorithm)

[5] [Topological Sorting instruction on GeeksforGeeks](http://www.geeksforgeeks.org/topological-sorting/)



